# Case A: Simple rainfall-runoff on an inclined plane

## Phenomenon
Models overland flow generated by rainfall on a sloping surface. Hydrological process for understanding runoff generation and surface water movement.

## Specific Setup
- Equations: Rainfall and friction active
- Terrain (z_terrain​): Simple inclined plane
- Initial Conditions (IC): Starts with some water present
- Boundary Conditions (BCs):
    - x=0, xmax: u=0
    - y=0, ymax: v=0


In [ ]:
# ----- Imports -----
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from mpl_toolkits.mplot3d import Axes3D
from IPython.display import HTML, display, clear_output
from tqdm.notebook import tqdm # Use notebook version if in Jupyter/Colab
from torch.optim.lr_scheduler import ExponentialLR
import time
import os

# Ensure reproducibility if desired (optional)
# torch.manual_seed(42)
# np.random.seed(42)

In [ ]:
import os

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
os.chdir("drive/My Drive/Colab Data/")
!ls

In [ ]:
# ----- Domain Definition & Simulation Parameters -----
domain = {'x_min': 0.0, 'x_max': 1.0, # Length of the slope
          'y_min': 0.0, 'y_max': 1.0, # Width of the slope
          't_min': 0.0, 't_max': 300.0} # Simulation time in seconds (e.g., 5 minutes)

# Sampling points
N_collocation = 10000 # Might need a decent number
N_bc = 2000          # For wall boundaries
N_ic = 2500

# Normalisation scaling factors
scale_x = 2.0 / (domain['x_max'] - domain['x_min'])
scale_y = 2.0 / (domain['y_max'] - domain['y_min'])
scale_t = 2.0 / (domain['t_max'] - domain['t_min'])

# --- Device Setup ---
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

In [ ]:
n_manning = 0.02  # Manning's roughness coefficient

In [ ]:
# --- Terrain Definition (Inclined Plane) ---
def create_terrain_torch(X, Y):
    """ Creates a simple inclined plane terrain. """
    if not isinstance(X, torch.Tensor): X = torch.tensor(X, dtype=torch.float32, device=device)
    if not isinstance(Y, torch.Tensor): Y = torch.tensor(Y, dtype=torch.float32, device=device)

    slope = 0.05  # Adjust slope (e.g., 0.05 means 5 units drop over 100 units length)
    z_max = 0.1   # Height at x=0
    terrain = z_max - slope * X
    return terrain

def create_terrain_np(x, y):
    """ Creates terrain using NumPy arrays by calling the torch version. """
    X_np, Y_np = np.meshgrid(x, y, indexing='xy')
    with torch.no_grad():
      terrain_torch = create_terrain_torch(torch.tensor(X_np, dtype=torch.float32, device=device),
                                           torch.tensor(Y_np, dtype=torch.float32, device=device))
    return terrain_torch.cpu().numpy()

# --- PINN Model Definition (Keep Same) ---
class FloodNet(nn.Module):
    def __init__(self, in_dim=3, hid_dim=64, out_dim=3): # Adjust hid_dim if needed
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_dim, hid_dim), nn.Tanh(),
            nn.Linear(hid_dim, hid_dim), nn.Tanh(),
            nn.Linear(hid_dim, hid_dim), nn.Tanh(),
            nn.Linear(hid_dim, hid_dim), nn.Tanh(),
            nn.Linear(hid_dim, out_dim)
        )
        self.init_weights()
    def init_weights(self):
      for m in self.modules():
          if isinstance(m, nn.Linear): nn.init.xavier_uniform_(m.weight); nn.init.zeros_(m.bias) if m.bias is not None else None
    def forward(self, x): return self.net(x)

# --- Coordinate Normalisation (Keep Same) ---
def normalize(x, y, t):
    x_n = scale_x * (x - domain['x_min']) - 1.0
    y_n = scale_y * (y - domain['y_min']) - 1.0
    t_n = scale_t * (t - domain['t_min']) - 1.0
    return x_n, y_n, t_n

In [ ]:
# --- Shallow Water Equations (Terrain Gradient Calculation) ---
def shallow_water_residuals(zeta, u, v, terrain_c, R_source, # Added R_source term
                            Xc, Yc, Tc, Xn, Yn, Tn,
                            g=9.81, friction_factor=0.01): # Default friction
    """ Calculates the residuals of the SWEs including rainfall source. """
    h = zeta - terrain_c
    h_safe = h + 1e-6

    # --- Calculate derivatives using autograd ---
    # Gradients w.r.t NORMALISED coordinates
    h_grads = torch.autograd.grad(h, (Xn, Yn, Tn), grad_outputs=torch.ones_like(h), create_graph=True); h_xhat, h_yhat, h_that = h_grads
    u_grads = torch.autograd.grad(u, (Xn, Yn, Tn), grad_outputs=torch.ones_like(u), create_graph=True); u_xhat, u_yhat, u_that = u_grads
    v_grads = torch.autograd.grad(v, (Xn, Yn, Tn), grad_outputs=torch.ones_like(v), create_graph=True); v_xhat, v_yhat, v_that = v_grads

    # Convert derivatives to PHYSICAL coordinates
    h_x, h_y, h_t = h_xhat * scale_x, h_yhat * scale_y, h_that * scale_t
    u_x, u_y, u_t = u_xhat * scale_x, u_yhat * scale_y, u_that * scale_t
    v_x, v_y, v_t = v_xhat * scale_x, v_yhat * scale_y, v_that * scale_t

    # --- Terrain slope derivatives ---
    # Terrain only depends on Xc in this scenario
    if Xc.requires_grad:
        z_x = torch.autograd.grad(terrain_c, Xc, grad_outputs=torch.ones_like(terrain_c), create_graph=True)[0]
    else: # Should not happen if called from train_pinn, but defensive check
        z_x = torch.zeros_like(terrain_c)

    # Since terrain does not depend on Yc, its gradient z_y is zero
    z_y = torch.zeros_like(z_x) # Create zeros with same shape/device as z_x
    # ------

    # Continuity residual
    r1 = h_t + u * h_x + h * u_x + v * h_y + h * v_y - R_source

    # Friction factors (to be used later on)
    # friction_u = -g * n_manning**2 * u[j, i] * np.sqrt(u[j, i]**2 + v[j, i]**2) / h[j, i]**(4/3)
    # friction_v = -g * n_manning**2 * v[j, i] * np.sqrt(u[j, i]**2 + v[j, i]**2) / h[j, i]**(4/3)

    # Momentum residuals
    r2 = u_t + u * u_x + v * u_y + g * (h_x + z_x) + friction_factor * u
    r3 = v_t + u * v_x + v * v_y + g * (h_y + z_y) + friction_factor * v # z_y is now correctly zero

    return r1, r2, r3

In [ ]:
# --- Training Loop (MODIFIED for Rainfall and Wall BCs) ---
def train_pinn(model, opt, scheduler, x_grid, y_grid, n_epochs=8000,
               w_pde=1.0, w_ic=150.0, w_bc=150.0, w_phys=15.0, # Loss weights
               friction_factor=0.01, R_const=1e-5, # Physics parameters (Rainfall in m/s)
               t_rain_start=0.0, t_rain_end=180.0): # Rainfall duration
    model.train(); loss_hist = []; start_time = time.time()

    # --- IC Setup (Near Zero) ---
    print("Setting up initial conditions (near-zero depth)..."); X0_np, Y0_np = np.meshgrid(x_grid, y_grid, indexing='xy'); T0_np = np.zeros_like(X0_np)
    terrain0_np = create_terrain_np(x_grid, y_grid); h0_np = np.full_like(X0_np, 1e-6); zeta0_np = h0_np + terrain0_np
    X0_t = torch.tensor(X0_np, dtype=torch.float32, device=device); Y0_t = torch.tensor(Y0_np, dtype=torch.float32, device=device)
    T0_t = torch.tensor(T0_np, dtype=torch.float32, device=device); zeta0_t = torch.tensor(zeta0_np, dtype=torch.float32, device=device)
    Xn0, Yn0, Tn0 = normalize(X0_t.unsqueeze(-1), Y0_t.unsqueeze(-1), T0_t.unsqueeze(-1))
    inp0 = torch.cat([Xn0, Yn0, Tn0], dim=2).view(-1, 3)
    # --- End IC Setup ---

    N_bc_each = N_bc // 4 # Equal points per wall boundary

    print(f"Starting training for {n_epochs} epochs...")
    for ep in tqdm(range(n_epochs), desc=f'Training Rainfall (R={R_const:.1e}, f={friction_factor:.3f})'):
        model.train(); opt.zero_grad()

        # --- PDE Loss ---
        x_c = torch.rand(N_collocation, 1, device=device)*domain['x_max']; y_c = torch.rand(N_collocation, 1, device=device)*domain['y_max']; t_c = torch.rand(N_collocation, 1, device=device)*domain['t_max']
        Xc=x_c.clone().detach().requires_grad_(True); Yc=y_c.clone().detach().requires_grad_(True); Tc=t_c.clone().detach().requires_grad_(True)
        terrain_c = create_terrain_torch(Xc, Yc)
        Xn,Yn,Tn = normalize(Xc,Yc,Tc); Xn=Xn.clone().detach().requires_grad_(True); Yn=Yn.clone().detach().requires_grad_(True); Tn=Tn.clone().detach().requires_grad_(True)
        inp_c = torch.cat([Xn,Yn,Tn], dim=1)
        zeta_c, u_c, v_c = model(inp_c).split(1,dim=1)

        # Define Rainfall Source Tensor based on sampled time Tc
        R_source_tensor = torch.where(
            (Tc >= t_rain_start) & (Tc <= t_rain_end),
            torch.full_like(Tc, R_const), # Constant rainfall during the period
            torch.zeros_like(Tc)          # Zero rainfall otherwise
        )

        r1,r2,r3 = shallow_water_residuals(zeta_c, u_c, v_c, terrain_c, R_source_tensor, # Pass R source
                                           Xc, Yc, Tc, Xn, Yn, Tn, friction_factor=friction_factor)

        # PDE_loss = torch.mean(r1**2) # Only use r1
        PDE_loss = torch.mean(r1**2) + torch.mean(r2**2) + torch.mean(r3**2)

        # --- Physical Loss ---
        h_c = zeta_c - terrain_c
        PHYS_loss = torch.mean(F.relu(-h_c)**2)

        # --- IC Loss ---
        zeta0_pred, u0_pred, v0_pred = model(inp0).split(1,dim=1)
        zeta0_pred_grid = zeta0_pred.view_as(X0_t)
        IC_loss = torch.mean((zeta0_pred_grid - zeta0_t)**2) + torch.mean(u0_pred**2) + torch.mean(v0_pred**2)

        # --- BC Loss (All Walls: u=0 or v=0) ---
        BC_loss = torch.tensor(0.0, device=device)
        t_bc = torch.rand(N_bc_each * 2, 1, device=device) * domain['t_max'] # Sample enough time points
        x_bc_wall = torch.rand(N_bc_each, 1, device=device) * domain['x_max']
        y_bc_wall = torch.rand(N_bc_each, 1, device=device) * domain['y_max']

        # BC x=0 (WALL: u=0)
        x0_bc_wall = torch.full_like(y_bc_wall, domain['x_min'])
        xn_w0, yn_w0, tn_w0 = normalize(x0_bc_wall, y_bc_wall, t_bc[:N_bc_each])
        inp_w0 = torch.cat([xn_w0, yn_w0, tn_w0], dim=1)
        _, u_w0, _ = model(inp_w0).split(1, dim=1)
        BC_loss += torch.mean(u_w0**2)

        # BC x=1 (WALL: u=0)
        x1_bc_wall = torch.full_like(y_bc_wall, domain['x_max'])
        xn_w1, yn_w1, tn_w1 = normalize(x1_bc_wall, y_bc_wall, t_bc[N_bc_each:N_bc_each*2]) # Use different time samples
        inp_w1 = torch.cat([xn_w1, yn_w1, tn_w1], dim=1)
        _, u_w1, _ = model(inp_w1).split(1, dim=1)
        BC_loss += torch.mean(u_w1**2)

        # BC y=0 (WALL: v=0)
        y0_bc_wall = torch.full_like(x_bc_wall, domain['y_min'])
        xn_wy0, yn_wy0, tn_wy0 = normalize(x_bc_wall, y0_bc_wall, t_bc[:N_bc_each])
        inp_wy0 = torch.cat([xn_wy0, yn_wy0, tn_wy0], dim=1)
        _, _, v_wy0 = model(inp_wy0).split(1, dim=1)
        BC_loss += torch.mean(v_wy0**2)

        # BC y=1 (WALL: v=0)
        y1_bc_wall = torch.full_like(x_bc_wall, domain['y_max'])
        xn_wy1, yn_wy1, tn_wy1 = normalize(x_bc_wall, y1_bc_wall, t_bc[N_bc_each:N_bc_each*2])
        inp_wy1 = torch.cat([xn_wy1, yn_wy1, tn_wy1], dim=1)
        _, _, v_wy1 = model(inp_wy1).split(1, dim=1)
        BC_loss += torch.mean(v_wy1**2)

        BC_loss /= 4.0 # Average the four wall losses

        # --- Total Loss & Backprop ---
        loss = w_pde*PDE_loss + w_ic*IC_loss + w_bc*BC_loss + w_phys*PHYS_loss
        loss.backward()
        opt.step()
        scheduler.step()
        # --- Logging ---
        if ep % 500 == 0 or ep == n_epochs - 1:
             loss_hist.append(loss.item())
             elapsed = time.time() - start_time
             print(f"Epoch {ep}/{n_epochs} | Loss: {loss.item():.4e} | PDE: {PDE_loss.item():.3e} | IC: {IC_loss.item():.3e} | BC: {BC_loss.item():.3e} | Phys: {PHYS_loss.item():.3e} | LR: {scheduler.get_last_lr()[0]:.2e} | Time: {elapsed:.1f}s")
    print(f"Training finished. Total time: {time.time() - start_time:.1f}s")
    return loss_hist

In [ ]:
# --- Visualisation Functions ---

# Visualisation Function (3D Surface Plot)
def visualize_results(model, x, y, t_vis, terrain_np, R_const, friction_factor, save_path="rainfall_3d.mp4"):
    print(f"Generating 3D surface animation (will save to {save_path})...")
    model.eval()
    Xp_np, Yp_np = np.meshgrid(x, y, indexing='xy')
    Xp_t = torch.tensor(Xp_np, dtype=torch.float32, device=device).unsqueeze(-1)
    Yp_t = torch.tensor(Yp_np, dtype=torch.float32, device=device).unsqueeze(-1)
    min_terrain = np.min(terrain_np); max_terrain = np.max(terrain_np)
    fig = plt.figure(figsize=(10, 8)); ax = fig.add_subplot(111, projection='3d')
    z_min_plot = min_terrain - 0.05 # Smaller buffer maybe
    z_max_plot_init = max_terrain + 0.1 # Initial guess for max water height
    ax.set_zlim(z_min_plot, z_max_plot_init); print(f"Initial 3D Z limits: ({z_min_plot:.2f}, {z_max_plot_init:.2f})")
    water_surf=None; max_zeta_observed=-np.inf; min_depth_display=1e-4 # Lower threshold maybe

    def update(frame):
        nonlocal water_surf,max_zeta_observed,z_max_plot_init
        ti=t_vis[frame]; ax.clear()
        with torch.no_grad():
            T_t=torch.full_like(Xp_t,fill_value=ti,dtype=torch.float32,device=device); Xn,Yn,Tn=normalize(Xp_t,Yp_t,T_t)
            inp=torch.cat([Xn,Yn,Tn],dim=2).view(-1,3); zeta_pred,_,_=model(inp).split(1,dim=1)
            zeta_np=zeta_pred.view(Xp_np.shape).cpu().numpy()
        max_zeta_observed=max(max_zeta_observed,np.max(zeta_np)); h_np=zeta_np-terrain_np
        zeta_display=np.where(h_np>min_depth_display,np.maximum(zeta_np,terrain_np+1e-5),np.nan)
        ax.plot_surface(Xp_np,Yp_np,terrain_np,color='dimgray',alpha=0.6,rstride=5,cstride=5)
        if not np.all(np.isnan(zeta_display)): water_surf=ax.plot_surface(Xp_np,Yp_np,zeta_display,color='skyblue',alpha=0.7,rstride=5,cstride=5)
        else: water_surf=None
        ax.set_title(f"Rainfall-Runoff 3D (R={R_const:.1e} m/s, f={friction_factor:.3f}): Time = {ti:.1f}s")
        ax.set_xlabel('X'); ax.set_ylabel('Y'); ax.set_zlabel('Elevation')
        current_z_max=max(z_max_plot_init,max_zeta_observed+0.05); ax.set_zlim(z_min_plot,current_z_max)

    ani=animation.FuncAnimation(fig,update,frames=len(t_vis),interval=100,blit=False) # Faster interval maybe
    try:
        print(f"Saving 3D animation to {save_path}..."); writer=animation.FFMpegWriter(fps=20,metadata=dict(artist='PINN Flood Sim'),bitrate=1800)
        ani.save(save_path,writer=writer); print(f"3D animation successfully saved to {save_path}")
    except Exception as e: print(f"\n--- ERROR: Failed to save 3D animation ---\n{e}"); save_path=None
    html_video=ani.to_jshtml(); plt.close(fig); print(f"Max zeta observed (3D): {max_zeta_observed:.3f}"); print("3D surface animation generation complete.")
    return HTML(html_video),save_path

# Visualisation Function (2D Velocity Plot)
def visualize_velocity_2d(model, x, y, t_vis, terrain_np, R_const, friction_factor, save_path="rainfall_2d.mp4"):
    print(f"Generating 2D velocity animation (will save to {save_path})...")
    model.eval()
    Xp_np, Yp_np = np.meshgrid(x, y, indexing='xy')
    Xp_t = torch.tensor(Xp_np, dtype=torch.float32, device=device).unsqueeze(-1)
    Yp_t = torch.tensor(Yp_np, dtype=torch.float32, device=device).unsqueeze(-1)
    fig,ax=plt.subplots(figsize=(10,8)); max_h_plot=0.05 # Small initial depth guess
    max_v_scale=0.2 # Small initial velocity guess
    skip=4; max_h_observed_anim=0.0; min_depth_display=1e-4
    terrain_contours = np.linspace(np.min(terrain_np), np.max(terrain_np), 8)

    def update_2d(frame):
        nonlocal max_h_plot,max_v_scale,max_h_observed_anim; ti=t_vis[frame]; ax.clear()
        with torch.no_grad():
            T_t=torch.full_like(Xp_t,fill_value=ti,dtype=torch.float32,device=device); Xn,Yn,Tn=normalize(Xp_t,Yp_t,T_t)
            inp=torch.cat([Xn,Yn,Tn],dim=2).view(-1,3); zeta_pred,u_pred,v_pred=model(inp).split(1,dim=1)
            zeta_np=zeta_pred.view(Xp_np.shape).cpu().numpy(); u_np=u_pred.view(Xp_np.shape).cpu().numpy(); v_np=v_pred.view(Xp_np.shape).cpu().numpy()
        h_np=zeta_np-terrain_np; h_plot=np.maximum(h_np,0); max_h_observed_anim=max(max_h_observed_anim,np.max(h_plot))
        if max_h_observed_anim > max_h_plot: max_h_plot=max_h_observed_anim # Dynamically adjust max depth for color scale
        depth_levels=np.linspace(min_depth_display,max_h_plot+1e-6,15)
        if len(depth_levels)>1: contour=ax.contourf(Xp_np,Yp_np,h_plot,levels=depth_levels,cmap='Blues',extend='max',alpha=0.8)
        else: contour=None
        quiver_mask=h_plot[::skip,::skip]>min_depth_display
        X_q=Xp_np[::skip,::skip][quiver_mask]; Y_q=Yp_np[::skip,::skip][quiver_mask]; U_q=u_np[::skip,::skip][quiver_mask]; V_q=v_np[::skip,::skip][quiver_mask]
        if X_q.size>0:
            current_max_v = np.max(np.sqrt(U_q**2+V_q**2)) if U_q.size>0 else 0.01
            if current_max_v > max_v_scale: max_v_scale = current_max_v # Dynamically adjust velocity scale
            dynamic_scale = max(max_v_scale,0.01)*30 # Adjust scale multiplier
            quiver = ax.quiver(X_q, Y_q, U_q, V_q, color='red', scale=dynamic_scale, scale_units='xy', angles='xy', width=0.003, headwidth=3, headlength=5)
        else: quiver=None
        ax.set_title(f"Rainfall-Runoff 2D (R={R_const:.1e} m/s, f={friction_factor:.3f}): Time = {ti:.1f}s")
        ax.set_xlabel("X"); ax.set_ylabel("Y"); ax.set_aspect('equal',adjustable='box'); ax.set_xlim(domain['x_min'],domain['x_max']); ax.set_ylim(domain['y_min'],domain['y_max'])
        ax.contour(Xp_np,Yp_np,terrain_np,levels=terrain_contours,colors='black',linewidths=0.6,alpha=0.4)

    ani_2d=animation.FuncAnimation(fig,update_2d,frames=len(t_vis),interval=100,blit=False)
    try:
        print(f"Saving 2D animation to {save_path}..."); writer=animation.FFMpegWriter(fps=20,metadata=dict(artist='PINN Flood Sim'),bitrate=1800)
        ani_2d.save(save_path,writer=writer); print(f"2D animation successfully saved to {save_path}")
    except Exception as e: print(f"\n--- ERROR: Failed to save 2D animation ---\n{e}"); save_path=None
    html_video_2d=ani_2d.to_jshtml(); plt.close(fig); print(f"Max depth observed (2D): {max_h_observed_anim:.3f}"); print("2D velocity animation generation complete.")
    return HTML(html_video_2d),save_path

In [ ]:
# --- Main Execution Block ---
if __name__ == '__main__':
    # --- Setup Grid and Time ---
    grid_res = 50
    xg = np.linspace(domain['x_min'], domain['x_max'], grid_res)
    yg = np.linspace(domain['y_min'], domain['y_max'], grid_res)
    t_vis_frames = 100 # More frames for longer sim
    tg_vis = np.linspace(domain['t_min'], domain['t_max'], t_vis_frames)
    terrain_np_vis = create_terrain_np(xg, yg)
    print(f"Inclined plane terrain created with min/max height: {np.min(terrain_np_vis):.3f}/{np.max(terrain_np_vis):.3f}")
    plt.figure(); plt.contourf(xg, yg, terrain_np_vis, levels=20); plt.colorbar(label="Elevation (m)"); plt.title("Inclined Plane Terrain"); plt.axis('equal'); plt.show()

    # --- Model, Optimiser, Scheduler ---
    model = FloodNet(in_dim=3, hid_dim=64, out_dim=3).to(device)
    learning_rate = 5e-4
    opt = torch.optim.Adam(model.parameters(), lr=learning_rate)
    scheduler = ExponentialLR(opt, gamma=0.997) # Maybe slower decay

    # --- Simulation Parameters ---
    n_epochs = 10000      # Adjust as needed
    friction_factor = 0.01 # Lower friction for thin flows? Tune this.
    # Rainfall Intensity (e.g., 50 mm/hr)
    rain_mm_hr = 50.0
    R_const = rain_mm_hr / (1000.0 * 3600.0) # Convert mm/hr to m/s
    t_rain_start = 0.0
    t_rain_end = 180.0 # Rain for first 3 minutes (180 seconds)
    # Loss Weights (tune these)
    weights = {'w_pde': 1.0, 'w_ic': 150.0, 'w_bc': 150.0, 'w_phys': 20.0} # Might need stronger physical constraint for wetting

    # --- Train the Model ---
    print("-" * 60); print(f"Starting training: Rainfall-Runoff Scenario"); print(f"Parameters: epochs={n_epochs}, lr={learning_rate}, friction={friction_factor}, R={R_const:.2e} m/s ({t_rain_start}-{t_rain_end}s)"); print(f"Weights: PDE={weights['w_pde']}, IC={weights['w_ic']}, BC={weights['w_bc']}, Phys={weights['w_phys']}"); print("-" * 60)
    loss_history = train_pinn(model, opt, scheduler, xg, yg, n_epochs=n_epochs,
                              friction_factor=friction_factor, R_const=R_const,
                              t_rain_start=t_rain_start, t_rain_end=t_rain_end, **weights)

    # --- Plot Loss History ---
    clear_output(wait=True)
    print("Training complete. Plotting loss history...")
    plt.figure(figsize=(8, 4)); plt.plot(np.arange(len(loss_history))*500, loss_history); plt.yscale('log'); plt.title('Training Loss History (Rainfall)'); plt.xlabel('Epoch'); plt.ylabel('Log Loss'); plt.grid(True, which='both', ls='--'); plt.show()

    # --- Generate, Save, and Display Visualisations ---
    save_name_3d = "rainfall_runoff_3d.mp4"
    save_name_2d = "rainfall_runoff_2d.mp4"

    html_3d, saved_3d_path = visualize_results(model, xg, yg, tg_vis, terrain_np_vis, R_const, friction_factor, save_path=save_name_3d)
    if saved_3d_path: print(f"3D Animation saved to: {os.path.abspath(saved_3d_path)}")
    display(html_3d)

    html_2d, saved_2d_path = visualize_velocity_2d(model, xg, yg, tg_vis, terrain_np_vis, R_const, friction_factor, save_path=save_name_2d)
    if saved_2d_path: print(f"2D Animation saved to: {os.path.abspath(saved_2d_path)}")
    display(html_2d)

    print("-" * 60); print("Script finished."); print("-" * 60)